## Skipgrams em Keras

- Nesta aula, implementaremos Skipgrams em `Keras`.

#### Carregando e pré-processando dados
- Carregar os dados de Alice no País das Maravilhas no Corpus usando o utilitário Keras
- `Keras` também possui alguns recursos úteis de pré-processamento de texto!
- Dividir o texto em sentenças.
- Usar o `Tokenizer` do `Keras` para tokenizar sentenças em palavras.

In [ ]:
# Imports
# Básicos
from __future__ import absolute_import, division, print_function  # Compatibilidade Python 2/3

import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np
import random
from IPython.display import SVG
%matplotlib inline

# nltk
from nltk import sent_tokenize

# keras
np.random.seed(13)
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Reshape, Activation
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import skipgrams

In [ ]:
# Vamos usar Alice no País das Maravilhas

path = get_file('carrol-alice.txt', origin="http://www.gutenberg.org/files/11/11-0.txt")
corpus = open(path).read()

In [ ]:
# Primeiro, dividir o documento em sentenças
corpus = corpus[corpus.index('\n\n')+2:]  # remover cabeçalho.
sentences = sent_tokenize(corpus)

# Tokenizar usando Keras
base_filter='!"#$%&()*+,-./:;`<=>?@[\\]^_{|}~\t\n' + "'"
tokenizer = Tokenizer(filters=base_filter)
tokenizer.fit_on_texts(sentences)

# Converter sentenças tokenizadas para formato de sequência
sequences = tokenizer.texts_to_sequences(sentences)
nb_samples = sum(len(s) for s in corpus)

print(len(sequences), tokenizer.document_count)

In [ ]:
# Para entender o que está acontecendo;

print(sentences[324])  # esta é uma sentença
print(sequences[324])  # esta é a mesma sentença onde as palavras são codificadas como números.
print(list(tokenizer.word_index[word.lower().replace('.', '')] 
           for word in sentences[324].split()))

#### Skipgrams: Gerando Rótulos de Entrada e Saída
- Agora que temos sentenças e tokenização de palavras, estamos em uma boa posição para criar nosso conjunto de treinamento para skipgrams.
- Agora precisamos gerar nosso `X_train` e `y_train`

In [ ]:
# Primeiro, vamos ver como a função skipgrams do Keras funciona.

couples, labels = skipgrams(sequences[324], len(tokenizer.word_index) + 1,
    window_size=2, negative_samples=0, shuffle=True,
    categorical=False, sampling_table=None)

index_2_word = {val: key for key, val in tokenizer.word_index.items()}

for w1, w2 in couples:
    if w1 == 13:
        print(index_2_word[w1], index_2_word[w2])

In [ ]:
# Função para gerar as entradas e saídas para todas as janelas

# Tamanho do vocabulário
vocab_size = len(tokenizer.word_index) + 1
# Dimensão para reduzir
dim = 100
window_size = 2


def generate_data(sequences, window_size, vocab_size):
    for seq in sequences:
        X, y = [], []
        couples, _ = skipgrams(
            seq, vocab_size,
            window_size=window_size, negative_samples=0, shuffle=True,
            categorical=False, sampling_table=None)
        if not couples:
            continue
        for in_word, out_word in couples:
            X.append(in_word)
            y.append(keras.utils.to_categorical(out_word, vocab_size))
        X, y = np.array(X), np.array(y)
        X = X.reshape(len(X), 1)
        y = y.reshape(len(X), vocab_size)
        yield X, y
        
data_generator = generate_data(sequences, window_size, vocab_size)

### Skipgrams: Criando o Modelo
- Por último, criamos a rede (rasa)!

In [ ]:
# Criar o modelo Keras e visualizá-lo 
skipgram = Sequential()
skipgram.add(Embedding(input_dim=vocab_size, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram.add(Reshape((dim,)))
skipgram.add(Dense(input_dim=dim, units=vocab_size, activation='softmax'))
# SVG(model_to_dot(skipgram, show_shapes=True).create(prog='dot', format='svg'))
# Nota: model_to_dot foi descontinuado, use plot_model se necessário visualizar

### Skipgrams: Compilando e Treinando
- Hora de compilar e treinar
- Usamos entropia cruzada, perda comum para classificação

In [ ]:
# Compilar o Modelo Keras
from tensorflow.keras.optimizers import SGD
sgd = SGD(learning_rate=1e-4, decay=1e-6, momentum=0.9)

skipgram.compile(loss='categorical_crossentropy', optimizer="adadelta")

# Treinar os Skipgrams
for iteration in range(10):
    loss = 0
    for x, y in generate_data(sequences, window_size, vocab_size):
        loss += skipgram.train_on_batch(x, y)
    print('iteração {}, perda é {}'.format(iteration, loss))

### Skipgrams: Observando os vetores

Para obter word_vectors agora, observamos os pesos da primeira camada.

Vamos também escrever funções que nos dão a similaridade de duas palavras.

In [ ]:
word_vectors = skipgram.get_weights()[0]


from scipy.spatial.distance import cosine


def get_dist(w1, w2):
    i1, i2 = tokenizer.word_index[w1], tokenizer.word_index[w2]
    v1, v2 = word_vectors[i1], word_vectors[i2]
    return cosine(v1, v2)

def get_similarity(w1, w2):
    return 1-get_dist(w1, w2)

def get_most_similar(w1, n=10):
    sims = {word: get_similarity(w1, word) 
            for word in tokenizer.word_index.keys()
            if word != w1}
    sims = pd.Series(sims)
    sims.sort_values(inplace=True, ascending=False)
    return sims.iloc[:n]


print(get_similarity('king', 'queen'))
print('')
print(get_most_similar('queen'))

## Sua vez -- Modifique o código acima para criar um Modelo CBOW